In [2]:
from process import event_preprocess, argument_preprocess,load_event, load_argument, Schema2Label
from src.seqlabel_utils import pos2bio
from src.preprocess.str_utils import get_useless_chars
from process import text_preprocess, gen_pos
from src.dataset.converter import single_text, double_text

In [8]:
train_e = load_event('./trainsample/duee_train.json')
valid_e = load_event('./trainsample/duee_dev.json')
test_e = load_event('./trainsample/duee_test2.json')
schema = Schema2Label('./trainsample/duee_event_schema.json')
train_e.shape, valid_e.shape, test_e.shape

((11908, 3), (1492, 3), (34904, 2))

## pipeline: EventType + Arguments

### 方案1：Trigger抽取序列标注问题

In [9]:
useless_chars = get_useless_chars(train_e['text'].values.tolist() +\
                                  valid_e['text'].values.tolist() +\
                                  test_e['text'].values.tolist())

In [23]:
train_e = event_preprocess(train_e, useless_chars)
valid_e = event_preprocess(valid_e, useless_chars)
test_e = event_preprocess(test_e, useless_chars)
valid_e.to_csv('./trainsample/valid_event.csv')
train_e.to_csv('./trainsample/train_event.csv')
test_e.to_csv('./trainsample/test_event.csv')

In [11]:
from src.dataset.converter import single_text
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_bio_label'].values,
            './trainsample','train_event_bio')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_bio_label'].values, 
            './trainsample','valid_event_bio')
single_text(test_e['id'].values, test_e['clean_text'].values, None ,'./trainsample','test_event_bio')

### 事件抽取方案2: 多标签分类问题

In [21]:
single_text(train_e['id'].values, train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event')
single_text(valid_e['id'].values, valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event')
single_text(test_e['id'].values, test_e['clean_text'].values, None, './trainsample','test_event')

### 事件抽取方案3：多标签分类 + 事件类别作为文本输入

In [41]:
enhancement = ''
pos = 1 
pos_list = [pos]

for i,j in enumerate(schema.event_label):
    event = j.replace('/','').replace('-','')
    enhancement+='[E{}]'.format(i) + event 
    pos+=len(event)+1
    pos_list.append(pos)


In [44]:
','.join([str(i) for i in pos_list])

'1,10,17,24,31,38,45,52,59,66,73,80,87,94,101,106,111,116,121,126,133,140,147,154,161,168,174,179,184,189,194,199,204,209,214,219,224,229,236,243,250,257,264,271,278,285,292,299,306,313,320,328,335,342,349,357,364,371,378,385,392,399,406,413,420,427'

In [35]:
double_text(train_e['id'].values, [enhancement] * train_e.shape[0], 
            train_e['clean_text'].values, train_e['event_label'].values,
            './trainsample','train_event_slot')
double_text(valid_e['id'].values, [enhancement] * valid_e.shape[0],
            valid_e['clean_text'].values, valid_e['event_label'].values, 
            './trainsample','valid_event_slot')
double_text(test_e['id'].values, [enhancement] * test_e.shape[0], 
            test_e['clean_text'].values, None, './trainsample','test_event_slot')

In [47]:
valid_e.head()

,id,text,trigger_list,clean_text,event_pos,event_bio_label,event_label
0,cba11b5059495e635b4f95e7484b2684,消失的“外企光环”，5月份在华裁员900余人，香饽饽变“臭”了,"[[组织关系-裁员, 裁员]]","消失的“外企光环”,5月份在华裁员900余人,香饽饽变“臭”了","[[组织关系-裁员, 15, 16]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[组织关系-裁员]
1,b90900665eee74f658eed125a321ee06,前两天，被称为 “ 仅次于苹果的软件服务商 ” 的 Oracle（ 甲骨文 ）公司突然宣布在...,"[[组织关系-裁员, 裁员]]","前两天,被称为“仅次于苹果的软件服务商”的Oracle(甲骨文)公司突然宣布在中国裁员。。","[[组织关系-裁员, 41, 42]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[组织关系-裁员]
2,c970d67bb8d3e57db77c4dbbdfbe9769,不仅仅是中国IT企业在裁员，为何500强的甲骨文也发生了全球裁员,"[[组织关系-裁员, 裁员], [组织关系-裁员, 裁员]]","不仅仅是中国IT企业在裁员,为何500强的甲骨文也发生了全球裁员","[[组织关系-裁员, 11, 12], [组织关系-裁员, 30, 31], [组织关系-裁...","[O, O, O, O, O, O, O, O, O, O, O, B-组织关系-裁员, I...","[组织关系-裁员, 组织关系-裁员]"
3,8a440ade8a8bac469e0357cc519ec9c0,据猛龙随队记者Josh Lewenberg报道，消息人士透露，猛龙已将前锋萨加巴-科纳特裁掉...,"[[组织关系-裁员, 裁掉], [组织关系-加盟, 签下], [组织关系-裁员, 裁掉]]","据猛龙随队记者JoshLewenberg报道,消息人士透露,猛龙已将前锋萨加巴-科纳特裁掉。...","[[组织关系-裁员, 43, 44], [组织关系-裁员, 71, 72], [组织关系-加...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[组织关系-裁员, 组织关系-加盟, 组织关系-裁员]"
4,80691881637ea5a0fee0ad01a689ce4a,冠军射手被裁掉，欲加入湖人队，但湖人却无意，冠军射手何去何从,"[[组织关系-裁员, 裁掉]]","冠军射手被裁掉,欲加入湖人队,但湖人却无意,冠军射手何去何从","[[组织关系-裁员, 5, 6]]","[O, O, O, O, O, B-组织关系-裁员, I-组织关系-裁员, O, O, O,...",[组织关系-裁员]


### Argument抽取

In [18]:
from process import process_stage1_test

In [22]:
train_a = argument_preprocess(train_a, useless_chars)
valid_a = argument_preprocess(valid_a, useless_chars)
test_a = argument_preprocess(test_a, useless_chars)

In [23]:
train_a.to_csv('./trainsample/train_argument.csv')
valid_a.to_csv('./trainsample/valid_argument.csv')
test_a.to_csv('./trainsample/test_argument.csv')

In [5]:
import ast
import pandas as pd 
valid = pd.read_csv('./trainsample/valid_event_cls_pred.csv')
valid['pred_label'] = valid['pred_label'].map(lambda x: ast.literal_eval(x))

In [ ]:
def multilabel_cls_report(y_true, y_pred):
    tp = 0
    fp = 0 
    fn = 0 
    for i,j in zip(y_true, y_pred):
        i,j = set(i),set(j)
        tp += len(i.intersection(j))
        fp += len(j.difference(i))
        fn += len(i.difference(j))
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy =
        
        
    
    

In [6]:
valid.head()

,Unnamed: 0,Unnamed: 0.1,id,text,trigger_list,text_clean,event_pos,event_bio_label,event_label,pred,prob,pred_label
0,0,0,cba11b5059495e635b4f95e7484b2684,消失的“外企光环”，5月份在华裁员900余人，香饽饽变“臭”了,"[['组织关系-裁员', '裁员']]","消失的“外企光环”,5月份在华裁员900余人,香饽饽变“臭”了","[['组织关系-裁员', 15, 16]]","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['组织关系-裁员'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0006579595501534641, 0.0006844821618869901,...",[组织关系-裁员]
1,1,1,b90900665eee74f658eed125a321ee06,前两天，被称为 “ 仅次于苹果的软件服务商 ” 的 Oracle（ 甲骨文 ）公司突然宣布在...,"[['组织关系-裁员', '裁员']]","前两天,被称为“仅次于苹果的软件服务商”的Oracle(甲骨文)公司突然宣布在中国裁员。。","[['组织关系-裁员', 41, 42]]","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['组织关系-裁员'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0006317550432868302, 0.0006702584796585143,...",[组织关系-裁员]
2,2,2,c970d67bb8d3e57db77c4dbbdfbe9769,不仅仅是中国IT企业在裁员，为何500强的甲骨文也发生了全球裁员,"[['组织关系-裁员', '裁员'], ['组织关系-裁员', '裁员']]","不仅仅是中国IT企业在裁员,为何500强的甲骨文也发生了全球裁员","[['组织关系-裁员', 11, 12], ['组织关系-裁员', 30, 31], ['组...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['组织关系-裁员', '组织关系-裁员']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0005977904656901956, 0.0006896447739563882,...",[组织关系-裁员]
3,3,3,8a440ade8a8bac469e0357cc519ec9c0,据猛龙随队记者Josh Lewenberg报道，消息人士透露，猛龙已将前锋萨加巴-科纳特裁掉...,"[['组织关系-裁员', '裁掉'], ['组织关系-加盟', '签下'], ['组织关系-...","据猛龙随队记者JoshLewenberg报道,消息人士透露,猛龙已将前锋萨加巴-科纳特裁掉。...","[['组织关系-裁员', 43, 44], ['组织关系-裁员', 71, 72], ['组...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['组织关系-裁员', '组织关系-加盟', '组织关系-裁员']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.004644543398171663, 0.0004959380603395402, ...","[组织关系-加盟, 组织关系-解雇]"
4,4,4,80691881637ea5a0fee0ad01a689ce4a,冠军射手被裁掉，欲加入湖人队，但湖人却无意，冠军射手何去何从,"[['组织关系-裁员', '裁掉']]","冠军射手被裁掉,欲加入湖人队,但湖人却无意,冠军射手何去何从","[['组织关系-裁员', 5, 6]]","['O', 'O', 'O', 'O', 'O', 'B-组织关系-裁员', 'I-组织关系...",['组织关系-裁员'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0029856592882424593, 0.00027657870668917894...",[组织关系-解雇]


In [3]:
valid_a.head()

,Unnamed: 0,Unnamed: 0.1,id,text,trigger_list,text_clean,event_pos,event_bio_label,event_label,pred,prob,pred_label,event_type
0,0,0,cba11b5059495e635b4f95e7484b2684,消失的“外企光环”，5月份在华裁员900余人，香饽饽变“臭”了,"[['组织关系-裁员', '裁员']]","消失的“外企光环”,5月份在华裁员900余人,香饽饽变“臭”了","[['组织关系-裁员', 15, 16]]","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['组织关系-裁员'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0006579595501534641, 0.0006844821618869901,...",组织关系-裁员,组织关系-裁员
1,1,1,b90900665eee74f658eed125a321ee06,前两天，被称为 “ 仅次于苹果的软件服务商 ” 的 Oracle（ 甲骨文 ）公司突然宣布在...,"[['组织关系-裁员', '裁员']]","前两天,被称为“仅次于苹果的软件服务商”的Oracle(甲骨文)公司突然宣布在中国裁员。。","[['组织关系-裁员', 41, 42]]","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",['组织关系-裁员'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0006317550432868302, 0.0006702584796585143,...",组织关系-裁员,组织关系-裁员
2,2,2,c970d67bb8d3e57db77c4dbbdfbe9769,不仅仅是中国IT企业在裁员，为何500强的甲骨文也发生了全球裁员,"[['组织关系-裁员', '裁员'], ['组织关系-裁员', '裁员']]","不仅仅是中国IT企业在裁员,为何500强的甲骨文也发生了全球裁员","[['组织关系-裁员', 11, 12], ['组织关系-裁员', 30, 31], ['组...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['组织关系-裁员', '组织关系-裁员']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0005977904656901956, 0.0006896447739563882,...",组织关系-裁员,组织关系-裁员
3,3,3,8a440ade8a8bac469e0357cc519ec9c0,据猛龙随队记者Josh Lewenberg报道，消息人士透露，猛龙已将前锋萨加巴-科纳特裁掉...,"[['组织关系-裁员', '裁掉'], ['组织关系-加盟', '签下'], ['组织关系-...","据猛龙随队记者JoshLewenberg报道,消息人士透露,猛龙已将前锋萨加巴-科纳特裁掉。...","[['组织关系-裁员', 43, 44], ['组织关系-裁员', 71, 72], ['组...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['组织关系-裁员', '组织关系-加盟', '组织关系-裁员']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.004644543398171663, 0.0004959380603395402, ...",组织关系-加盟,组织关系-加盟
4,3,3,8a440ade8a8bac469e0357cc519ec9c0,据猛龙随队记者Josh Lewenberg报道，消息人士透露，猛龙已将前锋萨加巴-科纳特裁掉...,"[['组织关系-裁员', '裁掉'], ['组织关系-加盟', '签下'], ['组织关系-...","据猛龙随队记者JoshLewenberg报道,消息人士透露,猛龙已将前锋萨加巴-科纳特裁掉。...","[['组织关系-裁员', 43, 44], ['组织关系-裁员', 71, 72], ['组...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['组织关系-裁员', '组织关系-加盟', '组织关系-裁员']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.004644543398171663, 0.0004959380603395402, ...",组织关系-解雇,组织关系-解雇


In [ ]:
def event_evaluation()

## MTL: EventType + Arugments

## Flatten: EventType * Arguments
